# Train Mask R-CNN Net for Object Detection

In [1]:
# import libraries
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from IPython.display import Image as disp_image
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os

## Mask R-CNN architecture

Mask R-CNN is a state of the art model for instance segmentation, developed on top of Faster R-CNN. Faster R-CNN is a region-based convolutional neural networks, that returns bounding boxes for each object and its class label with a confidence score.

Architecture of Mask R-CNN is based on Faster R-CNN that works in two stages:

``Stage1``: The first stage consists of two networks, backbone (ResNet, VGG, Inception, etc..) and region proposal network (RPN). These networks run once per image to give a set of region proposals. Region proposals are regions in the feature map which contain the object.

``Stage2``: In the second stage, the network predicts bounding boxes and object class for each of the proposed region obtained in stage1. Each proposed region can be of different size whereas fully connected layers in the networks always require fixed size vector to make predictions. Size of these proposed regions is fixed by using either RoI pool (which is very similar to MaxPooling) or RoIAlign method.

In [2]:
disp_image(url='https://developers.arcgis.com/python/guide/base64-images/how_maskrcnn_works-gd93ecb95-5e9d-28a9-6c68-1c7e0fa8e6b4.png', 
      width=550, height=400
)

## Region Proposal Network (RPN)

In object detection using R-CNN, RPN is the one true backbone and have proven to be very efficient till now. It’s purpose is to propose multiple objects that are identifiable within a particular image.

R-CNN is used to identify the objects but this algorithm specifically dwells into the logic and math behind how algorithm gets the box around the identified objects.

To generate these so called “proposals” for the region where the object lies, a small network is slide over a convolutional feature map that is the output by the last convolutional layer.


This ``object detection approach`` is to generate the candidate boxes (which might have our objects to detect) and classify those boxes as one of the objects. That is classification. At the same time, these box shape adjustments learn to properly fit the actual object. That is bounding box regression.

The first step, which is the generation of candidate boxes, is done by RPN. In the early versions of object detectors, this proposal generation happens offline by traditional computer vision techniques. One such approach is selective search. The drawbacks of these approaches are computation cost and also offline computation.

RPN came to the rescue by doing this in very little time and also it can be merged to any object detection network which makes it useful for end-to-end training. Just like how CNNs learn classification from feature maps, it also learns the proposals from feature maps. 

``Main steps in RPN``:
- Generate anchor boxes.
- Classify each anchor box whether it is foreground or background.
- Learn the shape offsets for anchor boxes to fit them for objects.

In [39]:
disp_image(url='https://www.mathworks.com/help/vision/ug/faster.png', 
      width=750, height=260
)

Every point in the feature map generated by the backbone network is an anchor point. Anchor boxes are needed to be generated for every anchor point. Candidate boxes are generated by using two parameters — scales and aspect ratios. The boxes need to be at image dimensions, whereas the feature map is reduced depending on the backbone. For example, in the case of vgg16, the image is reduced by 16 times by the end of the backbone. To generate boxes at image dimensions, this 16 is used as the stride in generating anchor boxes at the image level. (Ex: If anchor scales are [8,16,32] and ratios are [0.5,1,2] and stride is 16, the combination of these scales and ratios is used to generate 9 anchor boxes for each anchor point and then a stride of 16 over the image is applied to take the next anchor box.)

In [27]:
disp_image(url='https://davidstutz.de/wordpress/wp-content/uploads/2018/02/ren.jpg', 
      width=500, height=320      
)

Now the anchor boxes are generated, but these are dummy boxes that are different from the actual object of interest. Also, there might be many boxes which are not having any object in it. So it is needed to learn whether the given box is foreground or background, at the same time it is needed to learn the offsets for the foreground boxes to adjust for fitting the objects. These two tasks are achieved by two convolution layers on the feature map obtained from the backbone network. 

In [20]:
disp_image(url='https://ars.els-cdn.com/content/image/1-s2.0-S0957417418303907-gr1.jpg', 
      width=800, height=420
)

Offsets for ``x, y, w, h`` values are learned, where ``(x, y)`` is the center of the box, ``w`` and ``h`` are width and height. These offsets are learned by regression. <ins>For learning these scores and offsets, it is needed to have targets. These targets are generated by comparing the anchor boxes with ground truth boxes.</ins> This process is anchor target generation. In anchor target generation, the IOU of GT boxes with anchor boxes is calculated to check if it is fg/bg and then the difference in the coordinates are calculated as targets to be learned by the regressor. Then these targets are used as input for cross-entropy loss and smooth l1 loss.

Once these fg/bg scores and offsets are learned using convolution layers, some portions of fg and bg boxes are considered according to confidence scores. The offsets are applied to those boxes to get the actual ROIs to be processed further. This post-processing of anchor boxes using offsets is called proposal generation. These final proposals are propagated forward through the ROI pooling layer and fc layers.

To sum up, RPN has a classifier and a regressor. There is the concept introduced  as anchors. Anchor is the central point of the sliding window. For ZF Model which was an extension of AlexNet, the dimensions are 256-d and for VGG-16, it was 512-d. Classifier determines the probability of a proposal having the target object. Regression regresses the coordinates of the proposals. For any image, scale and aspect-ratio are two important parameters. (Aspect ratio = width of image / height of image, scale is the size of the image). The developers chose 3 scale and 3 aspect-ratio. So, total of 9 proposals are possible for each pixel, this is how the value of k is decided, K=9 for this case, k being the number of anchors. For the whole image, number of anchors is `W*H*K`.

This algorithm is robust against translations, therefore one of the key property of this algorithm it is translational invariant.

Presence of multi-scale anchors in the algorithm results in “Pyramid of Anchors” instead of “Pyramid of Filters” which makes it less time consuming and more cost efficient than previously proposed algorithms like Multi-Box.

These anchors are assigned label based on two factors:

1. The anchors with highest Intersection-over-union overlap with a ground truth box.
2. The anchors with Intersection-Over-Union Overlap higher than 0.7.

## Region of Interest (RoI)

A region of interest (often abbreviated ROI) is a sample within a data set identified for a particular purpose. The concept of a ROI is commonly used in many application areas. For example, in medical imaging, the boundaries of a tumor may be defined on an image or in a volume, for the purpose of measuring its size. In computer vision and optical character recognition, the ROI defines the borders of an object under consideration. In many applications, symbolic (textual) labels are added to a ROI, to describe its content in a compact manner. Within a ROI may lie individual points of interest (POIs).

In [6]:
disp_image(url='https://erdem.pl/static/2c35532f7b36e72f542981c094ed4d91/57882/fast-rcnn.png', 
      width=550, height=200
)

If original RoI is put on feature map it could not really apply the pooling layer because some of the “cells” are divided. What quantization is doing is that every result is rounded down before placing it on the matrix. 9.25 becomes 9, 4.53 becomes 4, etc. As a result, a bunch of data (dark blue) is lost and new data (green) is gained.

In [7]:
disp_image(url='https://erdem.pl/static/63acc1bc8437024bb0114e48cd1b3fae/e0885/box-cropped-losses.png', 
      width=400, height=400
)

Now when there is RoI mapped onto feature map we pooling can be applied on it. After RoI Pooling Layer there is a Fully Connected layer with a fixed size. Because RoIs have different sizes, they have to be pooled into the same size (3x3x512 in the example). At this moment the mapped RoI is a size of 4x6x512 and it cannot be divided 4 by 3. That’s where quantization strikes. (Quantization is a process of constraining an input from a large set of values (like real numbers) to a discrete set (like integers)).

In [8]:
disp_image(url='https://erdem.pl/static/e9c8da71e22237ae31c7e486f538e5ba/98432/box-cropped.png', 
      width=800, height=300
)

This process is done on the whole RoI matrix not only on the topmost layer. So the end result looks like this:

In [36]:
disp_image(url='https://erdem.pl/static/a0e28e647a77ad7e50d66b1c095a647a/4af8e/full-size-of-pooling.png', 
      width=250, height=180
)

The same process is applied to every single RoI from the original image so in the end, it might have hundreds or even thousands of 3x3x512 matrixes. Every one of those matrixes has to be sent through the rest of the network (starting from the FC layer). For each of them, the model is generating bbox and class separately.

In the second stage of Faster R-CNN, RoI pool is replaced by RoIAlign which helps preserve spatial information which gets misaligned in case of RoI pool. RoIAlign uses binary interpolation to create a feature map that is of fixed size.

The output from RoIAlign layer is then fed into Mask head, which consists of two convolution layers. It generates mask for each RoI, thus segmenting an image in pixel-to-pixel manner.

## 1. Initialize device and dataset

In [ ]:
# initialize parameters of data
BATCH_SIZE = 2
IMAGE_SIZE = [600, 600]

# define device
device = torch.device('cuda') if torch.cuda.is_available()

``IMAGE_SIZE = [Width, Height]`` are the dimensions of the image used for training. All images during the training processes will be resized to this size.

``BATCH_SIZE`` is the number of images that will be used for each iteration of the training.

``BATCH_SIZE * Width * Height`` will be proportional to the memory requirement of the training. Depending on your hardware, it might be necessary to use a smaller BATCH_SIZE or image size to avoid out-of-memory problems.

Note that since there is only a single image size, the net once trained is likely to be limited to work with only images of this size. In most cases what is necessary is to change the size of each training batch.

``device`` is automatically set the device where the net will run (GPU or CPU), in practice training without a strong GPU is extremely slow.

In [ ]:
# create a list of all images in the dataset
train_dir = 'LabPicsChemistry/Train'

imgs = []
for pth in os.listdir(train_dir):
    imgs.append(train_dir + '/' + pth + '//')

``train_dir`` is the LabPics V2 dataset train folder.

``imgs`` is the list of all images in the trainset

## 2. Create dataloader

Next, a data loader function is created that will allow for loading a batch of random images and their data for training. The data will contain the image and and masks of all the objects in the image. Each mask will be saved as a black-white (0/1) image.

These masks are images the same size as the RGB image where the region of the object instances is marked 1 and the rest are marked 0.

In [ ]:
# dataloader
def load_data():
  
    batch_imgs = []
    batch_data = []

    # loop over images in the batch  
    for i in range(BATCH_SIZE):
        
        # pick a random image from the list (idx)
        idx = random.randint(0, len(imgs) - 1)
        
        # load the image
        img = cv2.imread(os.path.join(imgs[idx], 'Image.jpg'))
        
        # resize image to the standard size (IMAGE_SIZE)
        img = cv2.resize(img, IMAGE_SIZE, cv2.INTER_LINEAR)
        
        # get subfolder where the vessel instances map is stored
        mask_dir = os.path.join(imgs[idx], 'Vessels')
        masks = []
        
        # loop over name of masks got from their directory
        for msk_name in os.listdir(mask_dir):
            
            # read the masks
            ves_mask = cv2.imread(mask_dir + '/' + msk_name, 0)
            
            # store mask in 0–255 format and is converted to 0–1 format
            ves_mask = (ves_mask > 0).astype(np.uint8) 
            
            # resize mask to the standard image size
            ves_mask = cv2.resize(ves_mask, IMAGE_SIZE, cv2.INTER_NEAREST)
            
            # add mask to the list
            masks.append(ves_mask)
        
        num_objs = len(masks)
        
        # test the number of objects on the image
        if num_objs == 0: 
            return load_data()
        
        boxes = torch.zeros([num_objs, 4], dtype=torch.float32)
        
        # use the masks to generate a bounding box for each object
        for i in range(num_objs):

            # x, y: are the top coordinate of the bounding box
            # w, h: are the width and height of the bounding box
            x, y, w, h = cv2.boundingRect(masks[i])

            # mask RCNN bounding box format demands the top left and bottom right
            # coordinate of the box which is given by: [x, y, x + w, y + h]
            boxes[i] = torch.tensor([x, y, x + w, y + h])
        
        # convert data into a tensor
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        img = torch.as_tensor(img, dtype=torch.float32)
        
        # stack all the information about the image into one dictionary
        data = {}
        data['boxes'] = boxes       
        data['masks'] = masks
        
        # pick ones for everything to take the class of all the objects to be the same (1)
        data['labels'] =  torch.ones((num_objs,), dtype=torch.int64)

        # add data to the lists
        batch_imgs.append(img)
        batch_data.append(data)  
  
  # load the image data into the training batch and convert it to PyTorch format
  batch_imgs = torch.stack([torch.as_tensor(d) for d in batch_imgs], 0)
  batch_imgs = batch_imgs.swapaxes(1, 3).swapaxes(2, 3)
  
  return batch_imags, batch_data

## 3. Train the model

Now start building the net. First, load a mask RCNN model that was already pretrained on the COCO dataset:

In [ ]:
# load pretreined model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

A pretrained model that uses existing knowledge can learn new tasks and datasets much faster than a model that was not trained before.

The COCO dataset contains over 100 classes. In this project, it is only needed to get two classes. There will be a change of the final layers of the net to predict two classes

In [ ]:
# get a number of input features in the head
in_features = model.roi_heads.box_predictor.cls_score.in_features 

# replace standard bounding box predictor with Fast-RCNN (2 classes) 
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

# load the model to the training device GPU or CPU
model.to(device)

In [ ]:
# define the optimizer to determine the way the net weights will be changed during training
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

In [ ]:
# set the model to train mode
model.train()

In [ ]:
# main training loop
for i in range(10001):
   
    # load the data using the data loader function
    images, targets = load_data()
   
    # load the data into the training device (CPU/GPU)
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
   
    # set gradients of all optimized tensors to zero
    optimizer.zero_grad()

    # take the images and data and run it through our neural net to get the loss
    loss_dict = model(images, targets)

    # loss is composed of several parts: class loss, bounding box loss, and mask loss; 
    # all of these parts are summed together to get the total loss as a single number
    losses = sum(loss for loss in loss_dict.values())
   
    # update the neural net weights using backpropagation
    losses.backward()
    optimizer.step()
    
    # print out losses
    print(i, 'loss:', losses.item())
   
    # save the trained model once every 500 steps
    if i % 200 == 0:
        torch.save(model.state_dict(), str(i) + '.torch')
        print('Save model to:', str(i) + '.torch')

## 4. Test the model

Once the training is finished the model has to be tested.

The script is similar to the training script. The first part is simply loading the net as before.

In [ ]:
# model for testing
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  

model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True) 

in_features = model.roi_heads.box_predictor.cls_score.in_features 
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

# the only difference is to load the saved model, 
# and set the model to evaluation state
model.load_state_dict(torch.load('10000.torch'))
model.to(device)
model.eval()

In [ ]:
# a single image loaded, resized to standard size, and converted to PyTorch format
images = cv2.imread(img_path)
images = cv2.resize(images, IMAGE_SIZE, cv2.INTER_LINEAR)
images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
images = images.swapaxes(1, 3).swapaxes(2, 3)
images = list(image.to(device) for image in images)

In [ ]:
# run the image through the net
with torch.no_grad():
    pred = model(images)

This runs the image through the net and gets a prediction for the object in the image. Note we are not training the net, so we do not need to collect gradient (no_grad) this makes the net run much faster.

The prediction is composed of several parts: “masks” which corresponds to the mask (regions) of every object in the image. “Scores” correspond to how likely the predicted mask is correct. In addition, there is the predicted bounding box and classes.

In [ ]:
# get image and its copy
im = images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.uint8)
im2 = im.copy()

# go over all the predictions 
# and display only those objects with “scores” larger than 0.8
for i in range(len(pred[0]['masks'])):
    
    msk = pred[0]['masks'][i, 0].detach().cpu().numpy()
    scr = pred[0]['scores'][i].detach().cpu().numpy()
    
    if scr > 0.8:
        im2[:, :, 0][msk > 0.5] = random.randint(0, 255)
        im2[:, :, 1][msk > 0.5] = random.randint(0, 255)
        im2[:, :, 2][msk > 0.5] = random.randint(0, 255)

cv2.imshow(str(scr), np.hstack([im, im2]))
cv2.waitKey()

Note that the predicted object ‘masks’ are saved as a matrix in the same size as the image with each pixel having a value that corresponds to how likely it is part of the object

Only pixels which values larger than 0.5 are likely to be part of the objects. This is displayed by marking these pixels with a different random color for each object